In [1]:
from matplotlib.pylab import *
from jax import lax, random, numpy as jnp
from flax import linen as nn
import numpy as np
from typing import Any, Callable, Optional, Tuple, Type, List
from jax import lax, random, numpy as jnp
import einops
import torchio
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import jax
# import monai_swin_nD
import tensorflow as tf
# import monai_einops
import torch 
import einops
import torchio as tio
import optax
from flax.training import train_state  # Useful dataclass to keep train state
from torch.utils.data import DataLoader
import h5py
import jax
from testUtils.spleenTest import get_spleen_data
from ml_collections import config_dict
from super_voxels.SIN.SIN_jax_3D.model_sin_jax_3D import SpixelNet
from swinTransformer.optimasation import get_optimiser
import swinTransformer.swin_transformer as swin_transformer
from swinTransformer.swin_transformer import SwinTransformer
from swinTransformer.losses import focal_loss
from swinTransformer.metrics import dice_metr
from swinTransformer.optimasation import get_optimiser
# import augmentations.simpleTransforms
# from augmentations.simpleTransforms import main_augment
from testUtils.spleenTest import get_spleen_data
from jax.config import config
from skimage.segmentation import mark_boundaries
import cv2
import functools
# from torch.utils.tensorboard import SummaryWriter
# import torchvision.transforms.functional as F
# import torchvision
import flax.jax_utils as jax_utils
import tensorflow as tf
from jax_smi import initialise_tracking
import ml_collections
import SimpleITK as sitk
import more_itertools
import toolz
import time
import SimpleITK as sitk
from super_voxels.SIN.SIN_jax_3D.render3D import *
from super_voxels.SIN.SIN_jax_3D.model_sin_jax_utils_3D import *




/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# # fourier_synthesis.py


# def calculate_2dft(input):
#     ft = np.fft.ifftshift(input)
#     ft = np.fft.fft2(ft)
#     return np.fft.fftshift(ft)

# def calculate_2dift(input):
#     ift = np.fft.ifftshift(input)
#     ift = np.fft.ifft2(ift)
#     ift = np.fft.fftshift(ift)
#     return ift.real

# def calculate_distance_from_centre(coords, centre):
#     # Distance from centre is √(x^2 + y^2)
#     return np.sqrt(
#         (coords[0] - centre) ** 2 + (coords[1] - centre) ** 2
#     )

# def find_symmetric_coordinates(coords, centre):
#     return (centre + (centre - coords[0]),
#             centre + (centre - coords[1]))

# def display_plots(individual_grating, reconstruction, idx):
#     plt.subplot(121)
#     plt.imshow(individual_grating)
#     plt.axis("off")
#     plt.subplot(122)
#     plt.imshow(reconstruction)
#     plt.axis("off")
#     plt.suptitle(f"Terms: {idx}")
#     plt.pause(0.01)

# # Read and process image
# image =get_spleen_data()[0][0][0,0,210:242,170:202,50]*256


# # Array dimensions (array is square) and centre pixel
# array_size = len(image)
# centre = int((array_size - 1) / 2)

# # Get all coordinate pairs in the left half of the array,
# # including the column at the centre of the array (which
# # includes the centre pixel)
# coords_left_half = (
#     (x, y) for x in range(array_size) for y in range(centre+1)
# )

# # Sort points based on distance from centre
# coords_left_half = sorted(
#     coords_left_half,
#     key=lambda x: calculate_distance_from_centre(x, centre)
# )
# # coords_left_half= coords_left_half[1:100]
# plt.set_cmap("gray")

# ft = calculate_2dft(image)

# # Show grayscale image and its Fourier transform
# plt.subplot(121)
# plt.imshow(image)
# plt.axis("off")
# plt.subplot(122)
# plt.imshow(np.log(abs(ft)))
# plt.axis("off")
# plt.pause(2)

# # Reconstruct image
# fig = plt.figure()
# # Step 1
# # Set up empty arrays for final image and
# # individual gratings
# rec_image = np.zeros(image.shape)
# individual_grating = np.zeros(
#     image.shape, dtype="complex"
# )
# idx = 0

# # All steps are displayed until display_all_until value
# display_all_until = 0
# # After this, skip which steps to display using the
# # display_step value
# display_step = 10
# # Work out index of next step to display
# next_display = display_all_until + display_step

# # Step 2
# for coords in coords_left_half:
#     # Central column: only include if points in top half of
#     # the central column
#     if not (coords[1] == centre and coords[0] > centre):
#         idx += 1
#         symm_coords = find_symmetric_coordinates(
#             coords, centre
#         )
#         # Step 3
#         # Copy values from Fourier transform into
#         # individual_grating for the pair of points in
#         # current iteration
#         individual_grating[coords] = ft[coords]
#         individual_grating[symm_coords] = ft[symm_coords]

#         # Step 4
#         # Calculate inverse Fourier transform to give the
#         # reconstructed grating. Add this reconstructed
#         # grating to the reconstructed image
#         rec_grating = calculate_2dift(individual_grating)
#         rec_image += rec_grating

#         # Clear individual_grating array, ready for
#         # next iteration
#         individual_grating[coords] = 0
#         individual_grating[symm_coords] = 0

#         # Don't display every step
#         if idx < display_all_until or idx == next_display:
#             if idx > display_all_until:
#                 next_display += display_step
#                 # Accelerate animation the further the
#                 # iteration runs by increasing
#                 # display_step
#                 display_step += 10
#             display_plots(rec_grating, rec_image, idx)

# plt.show()

In [3]:

cfg = config_dict.ConfigDict()
cfg.batch_size=1


def add_batches(cached_subj,cfg):
  cached_subj=list(more_itertools.chunked(cached_subj, cfg.batch_size))
  cached_subj=list(map(toolz.sandbox.core.unzip,cached_subj ))
  cached_subj=list(map(lambda inn: list(map(list,inn)),cached_subj ))
  cached_subj=list(map(lambda inn: list(map(np.concatenate,inn)),cached_subj ))
  return cached_subj


cached_subj =get_spleen_data()[0:43]
# cached_subj= add_batches(cached_subj,cfg)







spleen not in dataset


In [4]:
example_slic=cached_subj[1][2][0,:,:,:]
example_image=cached_subj[1][0]
ids = np.unique(np.ravel(np.array(example_slic)) )
ids
# ids = np.unique(np.ravel(np.array(example_slic)) )

array([0], dtype=uint32)

In [5]:
np.max(np.ravel(example_image))

0.6160167

In [6]:
# spacing = (1.5,1.5,1.5)
# slic = sitk.SLICImageFilter()
# # slic.SetMaximumNumberOfIterations(300)
# slic.SetEnforceConnectivity(True)
# # image_sitk_nda=einops.rearrange(image,'bb cc a b c -> (bb cc c) b a')
# image_sitk = sitk.GetImageFromArray(example_image)
# image_sitk.SetSpacing(spacing)

# rescalFilt=sitk.RescaleIntensityImageFilter()
# rescalFilt.SetOutputMaximum(1000)
# rescalFilt.SetOutputMinimum(0)
# image_sitk=rescalFilt.Execute(image_sitk)

# image_sitk=sitk.Cast(image_sitk, sitk.sitkInt64)
# slic_seg = slic.Execute(image_sitk)

# example_slic = sitk.GetArrayFromImage(slic_seg)
# ids = np.unique(np.ravel(np.array(example_slic)) )

In [7]:
ids

array([0], dtype=uint32)

In [8]:

curr_id=ids[10]
curr_example_slic=(example_slic==curr_id)
points=np.argwhere(curr_example_slic)
values=list(map(lambda point :example_image[point[0],point[1],point[2]], points))
np.var(example_image[curr_example_slic])


def getMinMax(a):
    a = list(a)
    return np.min(a), np.max(a)

x,y,z=toolz.sandbox.core.unzip(points)
min_x, max_x= getMinMax(x)
min_y, max_y= getMinMax(y)
min_z, max_z= getMinMax(z)
section= example_slic[min_x:max_x,min_y:max_y, min_z:max_z]
mediann=np.median(np.ravel(section))
print(f"curr_id {curr_id} mediann {mediann}")
print(f"min_x {min_x} max_x {max_x} min_y {min_y} max_y {max_y} min_z {min_z} max_z {max_z}")


IndexError: index 10 is out of bounds for axis 0 with size 1